# # SpringBoard Capstone2: Direct Marketing
## Unit16: Pre-processing and Training Data Development 
> Data from Kaggle.com: https://www.kaggle.com/c/bankdirectmarketing/data 

In [1]:
# pip install imbalanced-learn

In [2]:
# check version number
import imblearn
print(imblearn.__version__)
from imblearn.over_sampling import SMOTE
from collections import Counter

0.8.0


In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [4]:
market_pre = pd.read_csv('Market_explored.csv', index_col = 0)
print(market_pre.head())

          age           job   marital            education  default housing  \
RecordID                                                                      
13783      49        admin.  divorced  professional.course       no     yes   
23986      52      services   married          high.school  unknown     yes   
20663      46   blue-collar  divorced             basic.9y       no      no   
13958      26  entrepreneur    single          high.school      yes     yes   
28184      47        admin.    single    university.degree       no      no   

         loan   contact month day_of_week  ...  campaign  pdays  previous  \
RecordID                                   ...                              
13783     yes  cellular   aug         mon  ...         1    115         2   
23986      no  cellular   may         mon  ...         1    402         2   
20663      no  cellular   apr         wed  ...         1    999         1   
13958     yes  cellular   aug         fri  ...        28    9

In [5]:
cat = market_pre[['job', 'marital', 'education', 'default', 'housing', \
 'loan', 'contact', 'month', 'day_of_week', 'poutcome']]
market_cat = pd.get_dummies(cat,drop_first = True)
print(market_cat.shape)
market_cat.head()

(22500, 43)


,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
RecordID,,,,,,,,,,,,,,,,,,,,,
13783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
23986,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
20663,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13958,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
28184,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


In [6]:
num = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', \
             'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
market_num = market_pre[num]
market_all = market_cat.join(market_num)
market_all.head()

,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,...,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
RecordID,,,,,,,,,,,,,,,,,,,,,
13783,0,0,0,0,0,0,0,0,0,0,...,49,4457,1,115,2,1.4,92.479703,-35.498996,0.705058,4990.198481
23986,0,0,0,0,0,0,1,0,0,0,...,52,4797,1,402,2,-1.8,93.439161,-39.331320,4.245479,5144.563621
20663,1,0,0,0,0,0,0,0,0,0,...,46,169,1,999,1,-1.8,93.075000,-47.100000,1.445000,5099.100000
13958,0,1,0,0,0,0,0,0,0,0,...,26,376,28,999,0,1.4,93.444000,-36.100000,4.967000,5228.100000
28184,0,0,0,0,0,0,0,0,0,0,...,47,3033,1,252,4,-3.4,94.352376,-33.073620,1.208702,5025.420036


In [7]:
X = market_all
y = market_pre['subscribe']


In [8]:
over = SMOTE(sampling_strategy=0.4)
X, y = over.fit_resample(X, y)
counter = Counter(y)
print(counter)

Counter({'no': 19548, 'yes': 7819})


#### SMOTE
https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print (X_train.shape, y_test.shape)

(21893, 53) (5474,)


In [10]:
# # build scaler based on training data and apply it to test data to then also scale the test data
# import sklearn.preprocessing
# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train_scaled=scaler.transform(X_train)
# X_test_scaled=scaler.transform(X_test)
# X_train_scaled[0:1]

In [11]:
# scaler = preprocessing.MinMaxScaler().fit(X_train)
# X_train_scaled=scaler.transform(X_train)
# X_test_scaled=scaler.transform(X_test)
# X_train_scaled[0:1]

In [12]:
scaler = preprocessing.StandardScaler().fit(X_train[num])
X_train_scaled=scaler.transform(X_train[num])
X_test_scaled=scaler.transform(X_test[num])
X_train_scaled[0:1]

array([[ 0.36915609, -0.7376168 , -0.40891552,  0.71342043, -0.6973484 ,
         0.01438729, -0.48661854, -0.3837983 ,  0.62535764,  0.76906431]])

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Instantiate the logistic regression classifier: logreg
logreg = LogisticRegression()
# Fit it to the training data
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3616  293]
 [ 964  601]]
              precision    recall  f1-score   support

          no       0.79      0.93      0.85      3909
         yes       0.67      0.38      0.49      1565

    accuracy                           0.77      5474
   macro avg       0.73      0.65      0.67      5474
weighted avg       0.76      0.77      0.75      5474



### the result without smote()

Standard scaler on num: <br>
[[3897   41] <br>
 [ 514   48]] <br>
 
              precision    recall  f1-score   support

          no       0.88      0.99      0.93      3938
         yes       0.54      0.09      0.15       562

    accuracy                           0.88      4500
   macro avg       0.71      0.54      0.54      4500
weighted avg       0.84      0.88      0.84      4500
